In [16]:
import ipywidgets as widgets
from IPython.display import display, Image
from io import BytesIO
from PIL import Image as PILImage
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import os
import json

In [5]:
import ipywidgets

In [2]:
# Upload widget
upload = widgets.FileUpload(
    accept='image/*',
    multiple=False,
    description='Wgraj zdjęcie'
)

# Text input na liczbę luster
num_mirrors = widgets.Text(
    value='',
    placeholder='np. 20, 25, 88',
    description='Numery luster:',
    style={'description_width': 'initial'}
)

In [23]:
BASE_DIR = "/home/pgliwny/Praca/Computer_vision_for_MAGIC/"
mirror_points_path = os.path.join(BASE_DIR, "data/points_IRCam.json")
img_path = os.path.join(BASE_DIR, "data/data/2025/12/15/IRCamM1T20251215_122547M.jpg")

with open(mirror_points_path, 'r') as f:
    data = json.load(f)

def get_point_coords(mirror_id):
    return data['mirror_ids'][f'id_{mirror_id:03d}']

In [27]:
# Przycisk
button = widgets.Button(description='Przetwórz', button_style='primary')

# Output
output = widgets.Output()

def on_click(b):
    output.clear_output()
    with output:
        if not upload.value:
            print("Najpierw wgraj zdjęcie!")
            return

        # Odczyt obrazu
        # Dla ipywidgets 8.x:
        uploaded_file = upload.value[0]

        img_bytes = uploaded_file.content    # 8.x

        img = PILImage.open(BytesIO(img_bytes))

        mirrors_id_list = [int(x.strip()) for x in num_mirrors.value.split(",") if x.strip()]
        print(f"Załadowano obraz: {img.size}")
        
        for mirror_id in mirrors_id_list:
            points = get_point_coords(mirror_id)

        fig, ax = plt.subplots(1, 1, figsize=(10, 8))
        ax.imshow(img)

        for mirror_id in mirrors_id_list:
            points = get_point_coords(mirror_id)
            # === Polygon na podstawie 4 współrzędnych ===
            polygon = patches.Polygon(
                points,
                closed=True,
                linewidth=2,
                edgecolor='red',
                facecolor='red',
                alpha=0.3  # przezroczystość wypełnienia
            )
            ax.add_patch(polygon)

        plt.tight_layout()
        plt.show()

In [28]:
button.on_click(on_click)

display(widgets.VBox([upload, num_mirrors, button, output]))